# 1. Cargamos los datos de Pronostico ya predichos por el Modelo Machine Learning

In [61]:
# Leer el archivo Parquet en un DataFrame de pandas
df = pd.read_parquet('train_lstm_tail_50.parquet', engine='pyarrow')

In [2]:
# df.tail()

In [98]:
datos_genetic = df.head(20)

In [287]:
# datos_genetic

# 2. Cargamos el Modelo Ml Entrenado

In [6]:
import tensorflow as tf
# Opcion 1
# from tensorflow.keras.models import load_model
# modelo_lstm = load_model('modelo_lstm_14_var.h5')

# Opcion 2
# # Cargar el modelo LSTM guardado
modelo_lstm = tf.keras.models.load_model('modelo_lstm_14_var.h5')

# 3. Construimos el Algoritmo Evolutivo Paso a Paso

### PASO 1

### Función para generar los límites dinámicos basados en ±5 unidades 

In [7]:
import numpy as np
import pandas as pd
import numpy as np
import random

# Función para generar los límites dinámicos basados en ±5 unidades
def generate_dynamic_limits(row):
    """
    Genera X_min y X_max para una fila específica restando y sumando 5 unidades respectivamente.
    
    Parameters:
    row (array-like): Fila del DataFrame.
    
    Returns:
    X_min, X_max (arrays): Arrays con los límites mínimos y máximos.
    """
    X_min = row - 2
    X_max = row + 2
    return X_min, X_max

### PASO 2

### Función para generar un individuo

In [8]:
# Función para generar un individuo
def generate_individual(random, df, start_index, n_var_independientes):
    """
    Genera un individuo basado en 4 filas consecutivas del DataFrame,
    usando los límites dinámicos ±2 para cada valor.
    
    Parameters:
    random (Random): Generador aleatorio.
    df (DataFrame): DataFrame con los datos originales.
    start_index (int): Índice de inicio para seleccionar las 4 filas consecutivas.
    n_var_independientes (int): Número de variables independientes a considerar (columnas).
    
    Returns:
    individual (array): Array de tamaño (4, n_var_independientes) con los valores generados.
    """
    individual = []
    
    # Asegurarnos de trabajar solo con las primeras `n_var_independientes` columnas
    df_independientes = df.iloc[:, :n_var_independientes]
    
    for i in range(len(datos_genetic)):  # len(datos_genetic)  = 4 (numero de pasos atras que utilizaste para entrenar la LSTM)
        # Seleccionar la fila i (de start_index a start_index+3) con solo n_var_independientes columnas
        row = df_independientes.iloc[start_index + i].values
        
        # Generar límites dinámicos para la fila actual
        X_min, X_max = generate_dynamic_limits(row)

        # Imprimir los límites generados para depuración
        print(f"Fila {i} - X_min: {X_min}")
        print(f"Fila {i} - X_max: {X_max}")

        # Generar una fila del individuo respetando los límites
        random_row = [
            random.uniform(X_min[j], X_max[j]) for j in range(len(row))
        ]
        
        # Añadir la fila generada al individuo
        individual.append(random_row)
    
    return np.array(individual)

### Generar un individuo

In [19]:
# # Semilla para reproducibilidad
# random.seed(42)

# # Generar un individuo
# individuo = generate_individual(random, datos_genetic, start_index=0, n_var_independientes=14)
# individuo

### Paso 3 

### Función para generar la población mejorada con validaciones y depuración

In [10]:
# Función para generar la población mejorada con validaciones y depuración
def generate_population(n_individuos_poblacion, random, df, start_index, n_var_independientes):
    """
    Genera una población de individuos.
    
    Parameters:
    pop_size (int): Tamaño de la población(Numero de individuos para la poblacion)
    random (Random): Generador aleatorio.
    df (DataFrame): DataFrame con los datos originales.
    start_index (int): Índice de inicio para seleccionar las filas.
    n_var_independientes (int): Número de variables independientes.
    
    Returns:
    list: Lista de individuos generados.
    """
    population = []
    
    for i in range(n_individuos_poblacion):
        # Generar un individuo
        individual = generate_individual(random, df, start_index, n_var_independientes)
        
        # Validar que el individuo se generó correctamente
        print(f"\nIndividuo {i+1}:")
        print(individual)
        
        # Añadir el individuo a la población
        population.append(individual)
    
    # Imprimir la forma de la población generada
    print("\nTamaño de la población generada:", len(population))
    print("Dimensiones de cada individuo:", np.array(population).shape)
    
    return population

In [20]:
# # Generar la población
# poblacion = generate_population(n_individuos_poblacion=3, random=random, df=datos_genetic, start_index=0, n_var_independientes=14)
# poblacion

### Paso 4

### Funcion fitness, que ayude a evaluar los individuos generados

In [12]:
def fitness_function(candidates, args, modelo_lstm):
    fitness_values = []
    for candidate in candidates:
        # Ajustar la forma del individuo para que sea compatible con la entrada del modelo LSTM
        individuo_reshape = np.array(candidate).reshape(1, 4, 14)
        
        # Realizar la predicción usando el modelo LSTM
        prediccion = modelo_lstm.predict(individuo_reshape)
        
        # Extraer el valor de la predicción (consumo de combustible) y añadirlo a la lista de fitness
        fitness_values.append(prediccion[0][0])  # Asegúrate de que `prediccion` tenga la forma correcta
    return fitness_values

In [21]:
# # Generar la población inicial
# poblacion = generate_population(n_individuos_poblacion=3, random=random, df=datos_genetic, start_index=0, n_var_independientes=14)

# # Evaluar la función de fitness en la población generada
# fitness_scores = fitness_function(candidates=poblacion, args=None, modelo_lstm=modelo_lstm)
# print(f"Fitness scores: {fitness_scores}")

# 4. Codigo Final Algoritmo Evolutivo - Aplica 1 solo Caso - Consolidado

In [237]:
import pandas as pd
from random import Random
from time import time
from inspyred import ec
from random import Random
from time import time
import numpy as np

import tensorflow as tf

# # Cargar el modelo LSTM guardado
modelo_lstm = tf.keras.models.load_model('modelo_lstm_14_var.h5')

# Leer el archivo Parquet en un DataFrame de pandas
df = pd.read_parquet('train_lstm_tail_50.parquet', engine='pyarrow')
datos_genetic = df.head(4)

# Función para generar los límites dinámicos basados en ±5 unidades
def generate_dynamic_limits(row):
    """
    Genera X_min y X_max para una fila específica restando y sumando 5 unidades respectivamente.
    
    Parameters:
    row (array-like): Fila del DataFrame.
    
    Returns:
    X_min, X_max (arrays): Arrays con los límites mínimos y máximos.
    """
    X_min = row - 2
    X_max = row + 2
    
    return X_min, X_max

# Función para generar un individuo
def generate_individual(random, df, start_index, n_var_independientes):
    """
    Genera un individuo basado en 4 filas consecutivas del DataFrame,
    usando los límites dinámicos ±2 para cada valor.
    
    Parameters:
    random (Random): Generador aleatorio.
    df (DataFrame): DataFrame con los datos originales.
    start_index (int): Índice de inicio para seleccionar las 4 filas consecutivas.
    n_var_independientes (int): Número de variables independientes a considerar (columnas).
    
    Returns:
    individual (array): Array de tamaño (4, n_var_independientes) con los valores generados.
    """
    individual = []
    
    # Asegurarnos de trabajar solo con las primeras `n_var_independientes` columnas
    df_independientes = df.iloc[:, :n_var_independientes]
    
    for i in range(len(datos_genetic)):  # len(datos_genetic)  = 4 (numero de pasos atras que utilizaste para entrenar la LSTM)
        # Seleccionar la fila i (de start_index a start_index+3) con solo n_var_independientes columnas
        row = df_independientes.iloc[start_index + i].values
        
        # Generar límites dinámicos para la fila actual
        X_min, X_max = generate_dynamic_limits(row)

        # Imprimir los límites generados para depuración
        #print(f"Fila {i} - X_min: {X_min}")
        #print(f"Fila {i} - X_max: {X_max}")

        # Generar una fila del individuo respetando los límites
        random_row = [
            random.uniform(X_min[j], X_max[j]) for j in range(len(row))
        ]
        
        # Añadir la fila generada al individuo
        individual.append(random_row)
    
    return np.array(individual)

# individuo = generate_individual(random, datos_genetic, start_index=0, n_var_independientes=14)
# individuo

# Función para generar la población mejorada con validaciones y depuración
def generate_population(n_individuos_poblacion, random, df, start_index, n_var_independientes):
    """
    Genera una población de individuos.
    
    Parameters:
    pop_size (int): Tamaño de la población(Numero de individuos para la poblacion)
    random (Random): Generador aleatorio.
    df (DataFrame): DataFrame con los datos originales.
    start_index (int): Índice de inicio para seleccionar las filas.
    n_var_independientes (int): Número de variables independientes.
    
    Returns:
    list: Lista de individuos generados.
    """
    population = []
    
    for i in range(n_individuos_poblacion):
        # Generar un individuo
        individual = generate_individual(random, df, start_index, n_var_independientes)
        
        # Validar que el individuo se generó correctamente
        print(f"\nIndividuo {i+1}:")
        print(individual)
        
        # Añadir el individuo a la población
        population.append(individual)
    
    # Imprimir la forma de la población generada
    print("\nTamaño de la población generada:", len(population))
    print("Dimensiones de cada individuo:", np.array(population).shape)
    
    return population

# Generar la población
# poblacion = generate_population(n_individuos_poblacion=3, random=random, df=datos_genetic, start_index=0, n_var_independientes=14)
# poblacion

def fitness_function(candidates, args, modelo_lstm):
    fitness_values = []
    for candidate in candidates:
        # Ajustar la forma del individuo para que sea compatible con la entrada del modelo LSTM
        individuo_reshape = np.array(candidate).reshape(1, 4, 14)
        
        # Realizar la predicción usando el modelo LSTM
        prediccion = modelo_lstm.predict(individuo_reshape)
        
        # Extraer el valor de la predicción (consumo de combustible) y añadirlo a la lista de fitness
        fitness_values.append(prediccion[0][0])  # Asegúrate de que `prediccion` tenga la forma correcta
    return fitness_values

# # Generar la población inicial
# poblacion = generate_population(n_individuos_poblacion=3, random=random, df=datos_genetic, start_index=0, n_var_independientes=14)

# # # Evaluar la función de fitness en la población generada
# fitness_scores = fitness_function(candidates=poblacion, args=None, modelo_lstm=modelo_lstm)
# print(f"Fitness scores: {fitness_scores}")\

def run_genetic_algorithm(modelo_lstm, df, n_individuos_poblacion=5, generations=5):
    # Crear el generador de números aleatorios
    prng = Random()
    #prng.seed(time())

    # Número de variables independientes en cada individuo
    n_var_independientes = 14  
    start_index = 0  # Índice de inicio para seleccionar las filas en tu dataset
    
    # # Inicializar la población
    # population = generate_population(n_individuos_poblacion, prng, df, start_index, n_var_independientes)
    
    # Crear el objeto del algoritmo genético (GA)
    ea = ec.GA(prng)

    # Configurar la función generadora de individuos
    def generator(random, args):
        return generate_individual(prng, df, start_index, n_var_independientes)
    
    # Crear un contenedor global para almacenar los fitness de cada generación
    fitness_history = {}

    #Configurar la función de evaluación (fitness)
    def evaluator(candidates, args):
        # Evaluar el fitness de todos los candidatos (individuos)
        fitness_values = fitness_function(candidates, args, modelo_lstm)
        
        # Obtener el número de generación actual
        current_generation = len(fitness_history) # + 1
        print(f"Evaluando Generación {current_generation}...")  # Imprimir para verificar
        
        # Guardar los fitness de la generación actual
        fitness_history[current_generation] = fitness_values
        
        # Mostrar los fitness de la generación actual
        print(f"\nFitness de la Generación {current_generation}: {fitness_values}")
        
        return fitness_values

    # Configuración de operadores genéticos
    ea.variator = [
        ec.variators.uniform_crossover,  # Cruce uniforme
        ec.variators.gaussian_mutation   # Mutación gaussiana
    ]

    # Cambiar el método de reemplazo a generational_replacement
    ea.replacer = ec.replacers.generational_replacement
    
    # ea.replacer = ec.replacers.steady_state_replacement  # Reemplazo en estado estable
    ea.selector = ec.selectors.tournament_selection  # Selección por torneo

    # Configurar la terminación por número de generaciones
    ea.terminator = ec.terminators.generation_termination


    # **Aquí agregamos el mensaje antes de la evolución**
    print(f"Ejecutando {generations} generaciones...")
    
    # Ejecutar la evolución
    final_pop = ea.evolve(
        generator=generator,
        evaluator=evaluator,
        pop_size=n_individuos_poblacion,
        maximize=False, #Problema de Minimizacion 
        num_elites=1,
        max_generations=generations
    )

        
    # Obtener el mejor individuo
    best_individual = min(final_pop, key=lambda x: x.fitness)
    best_fitness = best_individual.fitness

    print("\nMejor individuo (minimizar consumo de combustible):", best_individual.candidate)
    print("Consumo de combustible predicho (fitness):", best_fitness)

    return best_individual.candidate, best_fitness

# # Asumiendo que tienes un modelo LSTM entrenado llamado `modelo_lstm`
best_individual, best_fitness = run_genetic_algorithm(modelo_lstm, datos_genetic, n_individuos_poblacion=5, generations=3)

Ejecutando 3 generaciones...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Evaluando Generación 0...

Fitness de la Generación 0: [80.42365, 70.182434, 207.88914, 291.48306, 160.89066]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Evaluando Generación 1...

Fitness de la Generación 1: [70.182434, 70.31807, 80.42365, 80.42365]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Evaluando Generación 2...

Fitness de la Generación 2: [70.182434, 70.182434, 70.182434, 69.4384]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Evaluando Generació

In [290]:
# best_individual, best_fitness = run_genetic_algorithm(modelo_lstm, resultados[4], n_individuos_poblacion=5, generations=3)

In [288]:
# best_individual[len(best_individual)-1]

In [289]:
# best_fitness

# 5. Codigo Final Algoritmo Evolutivo - Aplicar a Base de datos Total - Consolidado

In [284]:
import pandas as pd
from random import Random
from time import time
from inspyred import ec
from random import Random
from time import time
import numpy as np

import tensorflow as tf

# # Cargar el modelo LSTM guardado
modelo_lstm = tf.keras.models.load_model('modelo_lstm_14_var.h5')

# Leer el archivo Parquet en un DataFrame de pandas
# df = pd.read_parquet('train_lstm_tail_50.parquet', engine='pyarrow')
# datos_genetic = df.head(4)

datos_genetic = pd.read_parquet('train_lstm_tail_50.parquet', engine='pyarrow')

# Tamaño de la ventana
window_size = 4
# Lista para almacenar los subconjuntos
resultados = []
# Recorrer el DataFrame usando ventanas deslizantes
for start in range(len(datos_genetic)):
    # Crear un subconjunto del DataFrame
    subset = datos_genetic.iloc[start:start + window_size].copy()
    
    # Si el subconjunto tiene menos filas que el tamaño de la ventana, agregar filas con NaN
    if len(subset) < window_size:
        filas_faltantes = window_size - len(subset)
        filas_nan = pd.DataFrame(np.nan, index=range(filas_faltantes), columns=datos_genetic.columns)
        subset = pd.concat([subset, filas_nan], ignore_index=True)
    
    resultados.append(subset)


def generate_dynamic_limits(row):
    """
    Genera X_min y X_max para una fila específica restando y sumando 5 unidades respectivamente.
    
    Parameters:
    row (array-like): Fila del DataFrame.
    
    Returns:
    X_min, X_max (arrays): Arrays con los límites mínimos y máximos.
    """
    X_min = row - 2
    X_max = row + 2
    
    return X_min, X_max

# Función para generar un individuo
def generate_individual(random, df, start_index, n_var_independientes, ventana=window_size):
    """
    Genera un individuo basado en `ventana` filas consecutivas del DataFrame.
    
    Parameters:
    random (Random): Generador aleatorio.
    df (DataFrame): DataFrame con los datos originales.
    start_index (int): Índice de inicio para seleccionar las filas consecutivas.
    n_var_independientes (int): Número de variables independientes a considerar (columnas).
    ventana (int): Tamaño de la ventana (número de filas a seleccionar).
    
    Returns:
    individual (array): Array de tamaño `(ventana, n_var_independientes)` con los valores generados.
    """
    individual = []

    # Asegurarse de trabajar solo con las primeras `n_var_independientes` columnas
    df_independientes = df.iloc[:, :n_var_independientes]

    # Verificar que el índice de inicio + ventana no exceda el número de filas en df_independientes
    if start_index + ventana > len(df_independientes):
        raise IndexError(f"El índice de inicio más la ventana excede el número de filas disponibles en el DataFrame (inicio: {start_index}, ventana: {ventana}).")
    
    # Iterar a través de la ventana
    for i in range(ventana):
        row = df_independientes.iloc[start_index + i].values
        
        # Generar límites dinámicos para la fila actual
        X_min, X_max = generate_dynamic_limits(row)

        # Generar una fila del individuo respetando los límites
        random_row = [random.uniform(X_min[j], X_max[j]) for j in range(len(row))]
        
        # Añadir la fila generada al individuo
        individual.append(random_row)
    
    return np.array(individual)


# individuo = generate_individual(random, datos_genetic, start_index=0, n_var_independientes=14)
# individuo

# Función para generar la población mejorada con validaciones y depuración
def generate_population(n_individuos_poblacion, random, df, start_index, n_var_independientes):
    """
    Genera una población de individuos.
    
    Parameters:
    pop_size (int): Tamaño de la población(Numero de individuos para la poblacion)
    random (Random): Generador aleatorio.
    df (DataFrame): DataFrame con los datos originales.
    start_index (int): Índice de inicio para seleccionar las filas.
    n_var_independientes (int): Número de variables independientes.
    
    Returns:
    list: Lista de individuos generados.
    """
    population = []
    
    for i in range(n_individuos_poblacion):
        # Generar un individuo
        individual = generate_individual(random, df, start_index, n_var_independientes)
        
        # Validar que el individuo se generó correctamente
        print(f"\nIndividuo {i+1}:")
        print(individual)
        
        # Añadir el individuo a la población
        population.append(individual)
    
    # Imprimir la forma de la población generada
    print("\nTamaño de la población generada:", len(population))
    print("Dimensiones de cada individuo:", np.array(population).shape)
    
    return population

# Generar la población
# poblacion = generate_population(n_individuos_poblacion=3, random=random, df=datos_genetic, start_index=0, n_var_independientes=14)
# poblacion

def fitness_function(candidates, args, modelo_lstm):
    fitness_values = []
    for candidate in candidates:
        # Ajustar la forma del individuo para que sea compatible con la entrada del modelo LSTM
        individuo_reshape = np.array(candidate).reshape(1, 4, 14)
        
        # Realizar la predicción usando el modelo LSTM
        prediccion = modelo_lstm.predict(individuo_reshape)
        
        # Extraer el valor de la predicción (consumo de combustible) y añadirlo a la lista de fitness
        fitness_values.append(prediccion[0][0])  # Asegúrate de que `prediccion` tenga la forma correcta
    return fitness_values

# # Generar la población inicial
# poblacion = generate_population(n_individuos_poblacion=3, random=random, df=datos_genetic, start_index=0, n_var_independientes=14)

# # # Evaluar la función de fitness en la población generada
# fitness_scores = fitness_function(candidates=poblacion, args=None, modelo_lstm=modelo_lstm)
# print(f"Fitness scores: {fitness_scores}")\

# Función de evaluación y algoritmo genético
def run_genetic_algorithm(modelo_lstm, df, n_individuos_poblacion=5, generations=5, start_index=0, window_size=4):
    # Crear el generador de números aleatorios
    prng = Random()
    
    # Número de variables independientes en cada individuo
    n_var_independientes = 14  
    
    # Crear el objeto del algoritmo genético (GA)
    ea = ec.GA(prng)

    # Configurar la función generadora de individuos
    def generator(random, args):
        #return generate_individual(prng, df, start_index, n_var_independientes, window_size)
        return generate_individual(random, df, start_index=0, n_var_independientes=14, ventana=window_size)
    
    # Crear un contenedor global para almacenar los fitness de cada generación
    fitness_history = {}

    # Configurar la función de evaluación (fitness)
    def evaluator(candidates, args):
        # Evaluar el fitness de todos los candidatos (individuos)
        fitness_values = fitness_function(candidates, args, modelo_lstm)
        
        # Obtener el número de generación actual
        current_generation = len(fitness_history)
        print(f"Evaluando Generación {current_generation}...")
        
        # Guardar los fitness de la generación actual
        fitness_history[current_generation] = fitness_values
        
        # Mostrar los fitness de la generación actual
        print(f"\nFitness de la Generación {current_generation}: {fitness_values}")
        
        return fitness_values

    # Configuración de operadores genéticos
    ea.variator = [
        ec.variators.uniform_crossover,  # Cruce uniforme
        ec.variators.gaussian_mutation   # Mutación gaussiana
    ]

    # Cambiar el método de reemplazo a generational_replacement
    ea.replacer = ec.replacers.generational_replacement
    ea.selector = ec.selectors.tournament_selection  # Selección por torneo

    # Configurar la terminación por número de generaciones
    ea.terminator = ec.terminators.generation_termination

    # Ejecutar la evolución
    final_pop = ea.evolve(
        generator=generator,
        evaluator=evaluator,
        pop_size=n_individuos_poblacion,
        maximize=False, #Problema de Minimizacion 
        num_elites=1,
        max_generations=generations
    )

    # Obtener el mejor individuo
    best_individual = min(final_pop, key=lambda x: x.fitness)
    best_fitness = best_individual.fitness

    print("\nMejor individuo (minimizar consumo de combustible):", best_individual.candidate)
    print("Consumo de combustible predicho (fitness):", best_fitness)

    return best_individual.candidate, best_fitness


# Función para aplicar el algoritmo genético por bloques
def aplicar_algoritmo_genetico_por_bloques(modelo_lstm, resultados, window_size=4, n_individuos_poblacion=5, generations=3):
    # Nombres de columnas originales del DataFrame
    columnas_originales = resultados[0].columns.tolist()  # Aseguramos que incluya todas las columnas de resultados[0]
    
    # Crear un DataFrame vacío para almacenar los resultados
    df_resultados_final = pd.DataFrame(columns=columnas_originales) 
    
    # Iterar sobre los bloques en resultados con una ventana deslizante
    for i in range(len(resultados) - window_size + 1):  # Ajustamos el rango para no exceder el tamaño del DataFrame
        # Ejecutar el algoritmo genético en el bloque actual, pasando el índice de inicio como `start_index`
        best_individual, best_fitness = run_genetic_algorithm(
            modelo_lstm, resultados[i], n_individuos_poblacion=n_individuos_poblacion, generations=generations, start_index=i, window_size=window_size)
        
        # Extraer solo la última fila de best_individual (la última fila de la matriz 4x14)
        ultima_fila = best_individual[-1]  # Esto obtiene la última fila del array
        
        # Crear una nueva fila con los valores de ultima_fila y best_fitness
        nueva_fila = list(ultima_fila)  # Convertimos la última fila en una lista
        nueva_fila.append(best_fitness)  # Agregamos el best_fitness al final
        
        # Convertimos la fila en un DataFrame temporal
        df_temp = pd.DataFrame([nueva_fila], columns=columnas_originales) 
        
        # Anexamos la fila al DataFrame final
        df_resultados_final = pd.concat([df_resultados_final, df_temp], ignore_index=True)

        # Ajustar el índice del DataFrame final para que comience en window_size - 1
        df_resultados_final.index = range(window_size - 1, window_size - 1 + len(df_resultados_final))

    return df_resultados_final


# resultados -> lista de DataFrames que contiene tus bloques de datos
# modelo_lstm -> tu modelo LSTM ya entrenado
df_resultados_recomendacion = aplicar_algoritmo_genetico_por_bloques(modelo_lstm, resultados, window_size=4, n_individuos_poblacion=5, generations=3)

/tmp/ipykernel_7049/641713332.py:33: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  subset = pd.concat([subset, filas_nan], ignore_index=True)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 168ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Evaluando Generación 0...

Fitness de la Generación 0: [133.70633, 58.4036, 137.89163, 119.16576, 220.85811]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Evaluando Generación 1...

Fitness de la Generación 1: [58.4036, 84.53974, 61.071793, 58.4036]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Evaluando Generación 2...

Fitness de la Generación 2: [58.4036, 61.06345, 50.75852, 58.4036]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Evaluando Generación 3...

Fitness de la Generación 3: 

/tmp/ipykernel_7049/641713332.py:248: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_resultados_final = pd.concat([df_resultados_final, df_temp], ignore_index=True)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Evaluando Generación 0...

Fitness de la Generación 0: [80.73161, 121.138664, 67.7677, 114.7641, 124.74814]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Evaluando Generación 1...

Fitness de la Generación 1: [99.68243, 61.07044, 78.62234, 121.76426]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Evaluando Generación 2...

Fitness de la Generación 2: [82.1312, 68.24302, 61.07044, 61.07044]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Evaluando Generación 3...

Fitness de la Generación 3: [61.07044, 61.07044, 61.07044, 61.07044]

Mejor individuo (minimizar consumo de combustible): [[-2.99854353e-02 -

In [303]:
datos_genetic_original = pd.read_parquet('train_lstm_tail_50.parquet', engine='pyarrow')

In [301]:
datos_genetic_original.head(4)

,Roll,Intake Manifold Pressure,Intake Manifold #2 Pressure,Engine Speed,Actual Gear,x,y,z,efhcargado,efhvacio,distancia_ruta_pendiente_grados,Ambient Air Temperature,Brake Pump Pressure,Fuel Consumption Rate(lag1),Fuel Consumption Rate(target)
0,-0.185613,-0.031118,-0.227759,0.927779,0.254222,0.434750,0.463418,0.421462,2.069348,1.512208,-0.29231,0.125486,-0.305275,125.440002,281.940002
1,-0.638040,-0.020913,-0.098554,0.606561,1.292677,0.437891,0.463617,0.419659,2.069348,1.512208,-0.29231,0.128764,-0.282307,281.940002,0.000000
2,-2.030574,-0.054912,-0.621835,1.192444,1.292677,0.430850,0.463834,0.394201,2.069348,1.512208,-0.29231,0.125486,-0.113067,0.000000,0.000000
3,-0.758491,-0.055875,-0.641055,1.000166,1.811904,0.422928,0.464084,0.342244,2.069348,1.512208,-0.29231,0.124502,-0.291575,0.000000,362.890015


In [294]:
df_resultados_final_v1.head(2)

,Roll,Intake Manifold Pressure,Intake Manifold #2 Pressure,Engine Speed,Actual Gear,x,y,z,efhcargado,efhvacio,distancia_ruta_pendiente_grados,Ambient Air Temperature,Brake Pump Pressure,Fuel Consumption Rate(lag1),Fuel Consumption Rate(target)
3,-2.420053,0.237574,-1.500637,-0.493422,1.043667,0.462832,1.453585,1.433739,1.971443,2.514955,1.150659,1.673854,-0.869451,-2.155465,50.758518
4,-0.662276,0.593767,-0.343381,1.407459,1.310699,-1.202171,-1.114944,-0.103127,1.931512,-0.429992,-2.195074,0.669925,1.698723,362.235742,61.070438


In [298]:
# Sumar los valores a partir de la fila 3 (es decir, desde el índice 3 en adelante)
consumo_combustible_real = datos_genetic_original.iloc[3:]['Fuel Consumption Rate(target)'].sum()
consumo_combustible_recomendacion =df_resultados_final_v1['Fuel Consumption Rate(target)'].sum()

print('Consumo de Combustible Real al final del turno', consumo_combustible_real )
print('Consumo de Combustible Con Recomendaciones al final del turno', consumo_combustible_recomendacion)

Consumo de Combustible Real al final del turno 7498.25
Consumo de Combustible Con Recomendaciones al final del turno 2712.2305


In [297]:
df_resultados_final_v1['Fuel Consumption Rate(target)'].sum()

2712.2305

# 6. Codigo Final Algoritmo Evolutivo - Aplicar a Base de datos Total - Reestricciones Variables especificas(Rangos) y Variables Fijas

In [333]:
df_prueba = datos_genetic_original.head(4)

In [334]:
df_prueba

,Roll,Intake Manifold Pressure,Intake Manifold #2 Pressure,Engine Speed,Actual Gear,x,y,z,efhcargado,efhvacio,distancia_ruta_pendiente_grados,Ambient Air Temperature,Brake Pump Pressure,Fuel Consumption Rate(lag1),Fuel Consumption Rate(target)
0,-0.185613,-0.031118,-0.227759,0.927779,0.254222,0.434750,0.463418,0.421462,2.069348,1.512208,-0.29231,0.125486,-0.305275,125.440002,281.940002
1,-0.638040,-0.020913,-0.098554,0.606561,1.292677,0.437891,0.463617,0.419659,2.069348,1.512208,-0.29231,0.128764,-0.282307,281.940002,0.000000
2,-2.030574,-0.054912,-0.621835,1.192444,1.292677,0.430850,0.463834,0.394201,2.069348,1.512208,-0.29231,0.125486,-0.113067,0.000000,0.000000
3,-0.758491,-0.055875,-0.641055,1.000166,1.811904,0.422928,0.464084,0.342244,2.069348,1.512208,-0.29231,0.124502,-0.291575,0.000000,362.890015


In [354]:
datos_genetic = pd.read_parquet('train_lstm_tail_50.parquet', engine='pyarrow')

# 3. Definir el Tamaño de la ventana
window_size = 4

# Lista para almacenar los subconjuntos de datos que iran entregando al Algoritmo Evolutivo
resultados = []
# Recorrer el DataFrame usando ventanas deslizantes
for start in range(len(datos_genetic)):
    # Crear un subconjunto del DataFrame
    subset = datos_genetic.iloc[start:start + window_size].copy()
    
    # Si el subconjunto tiene menos filas que el tamaño de la ventana, agregar filas con NaN
    if len(subset) < window_size:
        filas_faltantes = window_size - len(subset)
        filas_nan = pd.DataFrame(np.nan, index=range(filas_faltantes), columns=datos_genetic.columns)
        subset = pd.concat([subset, filas_nan], ignore_index=True)
    
    resultados.append(subset)

# 4. Define los límites dinámicos para las columnas del DataFrame
config_2 = {
    "dynamic_limits": {
        "Roll": (-1, 1),    # Limitar var1 a ±2 del valor original
        "Intake Manifold Pressure": (-1, 1), 
        "Intake Manifold #2 Pressure": (-1, 1),    # Limitar var1 a ±2 del valor original
        "Engine Speed": (-1, 1), 
        "Actual Gear": (-1, 1),    # Limitar var1 a ±2 del valor original
        "x": "fixed" , 
        "y": "fixed" ,    # Limitar var1 a ±2 del valor original
        "z": "fixed" , 
        "efhcargado": (-1, 1),    # Limitar var1 a ±2 del valor original
        "efhvacio": (-1, 1), 
        "distancia_ruta_pendiente_grados": (-1, 1),    # Limitar var1 a ±2 del valor original
        "Ambient Air Temperature": (-1, 1), 
        "Brake Pump Pressure": (-1, 1),    # var3 es fija (sin modificaciones)\
        "Fuel Consumption Rate(lag1)": "fixed"    # Limitar var1 a ±2 del valor original
    }
}


def generate_dynamic_limits(row, config):
    """
    Genera X_min y X_max para una fila específica basada en configuraciones personalizadas.
    
    Parameters:
    row (array-like): Fila del DataFrame.
    config (dict): Configuración de límites dinámicos por columna.
    
    Returns:
    X_min, X_max (arrays): Arrays con los límites mínimos y máximos.
    """
    #Estas listas se llenarán con los límites mínimos (X_min) y máximos (X_max) para cada valor en la fila procesada
    X_min = []
    X_max = []
    
    for i, value in enumerate(row): # enumerate, nos permite iterar sobre los valores de la fila (row) y obtener tanto el índice i como el valor value
        variable_name = config["column_names"][i]  # config["column_names"] contiene los nombres de las columnas en el orden en que aparecen en el DataFrame.
        
        # Si el nombre de la columna tiene la etiqueta "fixed" en config["dynamic_limits"], la variable es fija y no debe modificarse
        if config["dynamic_limits"].get(variable_name) == "fixed":  # 
            # Variable fija, los límites son el mismo valor
            X_min.append(value)
            X_max.append(value)
        else:
            # Aplicar los límites dinámicos definidos
            # Obtiene los valores lower (límite inferior) y upper (límite superior) definidos en el diccionario de configuración para la columna actual (variable_name).
            lower, upper = config["dynamic_limits"][variable_name]
            X_min.append(value + lower)
            X_max.append(value + upper)
    
    return np.array(X_min), np.array(X_max)


def generate_individual(random, df, start_index, n_var_independientes, ventana, config):
    """
    Genera un individuo basado en configuraciones personalizadas.
    
    Parameters:
    random (Random): Generador aleatorio.
    df (DataFrame): DataFrame con los datos originales.
    start_index (int): Índice de inicio para seleccionar las filas consecutivas.
    n_var_independientes (int): Número de variables independientes a considerar (columnas).
    ventana (int): Tamaño de la ventana (número de filas a seleccionar).
    config (dict): Configuración personalizada para los límites dinámicos.
    
    Returns:
    individual (array): Array de tamaño `(ventana, n_var_independientes)` con los valores generados.
    """
    individual = []

    # Asegurarse de trabajar solo con las primeras `n_var_independientes` columnas(No debemos incluir la columna Target: En este caso la ultima columna)
    df_independientes = df.iloc[:, :n_var_independientes]  # Selecciona únicamente las primeras n_var_independientes columnas del DataFrame 
    config_2["column_names"] = df_independientes.columns.tolist() # Guarda los nombres de estas columnas en config["column_names"] para usarlos más adelante en la generación de límites dinámicos.

    # Verificar que el índice de inicio + ventana no exceda el número de filas en df_independientes
    if start_index + ventana > len(df_independientes):
        raise IndexError(f"El índice de inicio más la ventana excede el número de filas disponibles en el DataFrame (inicio: {start_index}, ventana: {ventana}).")
    
    # Iterar a través de la ventana
    for i in range(ventana): # Itera sobre las filas consecutivas del DataFrame, comenzando desde start_index
        row = df_independientes.iloc[start_index + i].values
        
        # Generar límites dinámicos para la fila actual
        X_min, X_max = generate_dynamic_limits(row, config = config_2)

        # Generar una fila del individuo respetando los límites
        random_row = [random.uniform(X_min[j], X_max[j]) for j in range(len(row))]
        
        # Añadir la fila generada al individuo
        individual.append(random_row)
    
    return np.array(individual)

# individuo = generate_individual(random, df_prueba, start_index=0, n_var_independientes=14, ventana=window_size, config = config )
# individuo

/tmp/ipykernel_7049/3033078186.py:17: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  subset = pd.concat([subset, filas_nan], ignore_index=True)


In [363]:
resultados[3]

,Roll,Intake Manifold Pressure,Intake Manifold #2 Pressure,Engine Speed,Actual Gear,x,y,z,efhcargado,efhvacio,distancia_ruta_pendiente_grados,Ambient Air Temperature,Brake Pump Pressure,Fuel Consumption Rate(lag1),Fuel Consumption Rate(target)
3,-0.758491,-0.055875,-0.641055,1.000166,1.811904,0.422928,0.464084,0.342244,2.069348,1.512208,-0.29231,0.124502,-0.291575,0.000000,362.890015
4,0.587039,0.033894,-0.033952,1.163037,1.811904,0.428635,0.464248,0.345397,2.069348,1.512208,-0.29231,0.123301,-0.290366,362.890015,412.940002
5,1.303871,0.264856,2.178688,0.516077,1.811904,0.445958,0.464304,0.383466,2.069348,1.512208,-0.29231,0.119039,-0.281702,412.940002,472.500000
6,1.124663,0.221559,1.792687,1.116287,1.292677,0.452901,0.464494,0.440137,2.069348,1.512208,-0.29231,0.111282,-0.281904,472.500000,203.789993


In [364]:
individuo = generate_individual(random, resultados[3], start_index=0, n_var_independientes=14, ventana=window_size, config = config_2)
individuo

array([[-4.50872713e-02,  4.47718961e-01, -3.48435607e-01,
         6.01941735e-01,  2.66563350e+00,  4.22927687e-01,
         4.64084312e-01,  3.42244122e-01,  1.95805357e+00,
         2.49010252e+00, -1.20484773e+00,  5.16008045e-02,
        -9.29748307e-01,  0.00000000e+00],
       [ 1.30198520e+00, -8.65839239e-01,  2.22189148e-01,
         1.22787959e+00,  1.19579964e+00,  4.28635444e-01,
         4.64247815e-01,  3.45396964e-01,  1.57912689e+00,
         1.73027086e+00,  2.57613834e-01, -5.80479127e-01,
        -1.13758292e-02,  3.62890015e+02],
       [ 1.63568109e+00, -1.48982590e-01,  2.28038524e+00,
        -1.86145416e-01,  1.34424249e+00,  4.45958100e-01,
         4.64303992e-01,  3.83466444e-01,  1.38306944e+00,
         2.37080790e+00,  1.13521816e-01,  3.91387918e-01,
         5.90645369e-01,  4.12940002e+02],
       [ 3.16209685e-01,  1.15747503e+00,  2.56026405e+00,
         1.68545449e+00,  3.88004821e-01,  4.52901051e-01,
         4.64493778e-01,  4.40137072e-01,  2.

In [366]:
import pandas as pd
from random import Random
from time import time
from inspyred import ec
from random import Random
from time import time
import numpy as np
import tensorflow as tf

# # 1. Cargar el modelo LSTM guardado
modelo_lstm = tf.keras.models.load_model('modelo_lstm_14_var.h5')

# 2. Leer el archivo Parquet en un DataFrame de pandas
# df = pd.read_parquet('train_lstm_tail_50.parquet', engine='pyarrow')
# datos_genetic = df.head(4)
datos_genetic = pd.read_parquet('train_lstm_tail_50.parquet', engine='pyarrow')

# 3. Definir el Tamaño de la ventana
window_size = 4

# Lista para almacenar los subconjuntos de datos que iran entregando al Algoritmo Evolutivo
resultados = []
# Recorrer el DataFrame usando ventanas deslizantes
for start in range(len(datos_genetic)):
    # Crear un subconjunto del DataFrame
    subset = datos_genetic.iloc[start:start + window_size].copy()
    
    # Si el subconjunto tiene menos filas que el tamaño de la ventana, agregar filas con NaN
    if len(subset) < window_size:
        filas_faltantes = window_size - len(subset)
        filas_nan = pd.DataFrame(np.nan, index=range(filas_faltantes), columns=datos_genetic.columns)
        subset = pd.concat([subset, filas_nan], ignore_index=True)
    
    resultados.append(subset)

# 4. Define los límites dinámicos para las columnas del DataFrame
config_2 = {
    "dynamic_limits": {
        "Roll": (-1, 1),    # Limitar var1 a ±2 del valor original
        "Intake Manifold Pressure": (-1, 1), 
        "Intake Manifold #2 Pressure": (-1, 1),    # Limitar var1 a ±2 del valor original
        "Engine Speed": (-1, 1), 
        "Actual Gear": (-1, 1),    # Limitar var1 a ±2 del valor original
        "x": "fixed" , 
        "y": "fixed" ,    # Limitar var1 a ±2 del valor original
        "z": "fixed" , 
        "efhcargado": (-1, 1),    # Limitar var1 a ±2 del valor original
        "efhvacio": (-1, 1), 
        "distancia_ruta_pendiente_grados": (-1, 1),    # Limitar var1 a ±2 del valor original
        "Ambient Air Temperature": (-1, 1), 
        "Brake Pump Pressure": (-1, 1),    # var3 es fija (sin modificaciones)\
        "Fuel Consumption Rate(lag1)": "fixed"    # Limitar var1 a ±2 del valor original
    }
}


def generate_dynamic_limits(row, config):
    """
    Genera X_min y X_max para una fila específica basada en configuraciones personalizadas.
    
    Parameters:
    row (array-like): Fila del DataFrame.
    config (dict): Configuración de límites dinámicos por columna.
    
    Returns:
    X_min, X_max (arrays): Arrays con los límites mínimos y máximos.
    """
    #Estas listas se llenarán con los límites mínimos (X_min) y máximos (X_max) para cada valor en la fila procesada
    X_min = []
    X_max = []
    
    for i, value in enumerate(row): # enumerate, nos permite iterar sobre los valores de la fila (row) y obtener tanto el índice i como el valor value
        variable_name = config["column_names"][i]  # config["column_names"] contiene los nombres de las columnas en el orden en que aparecen en el DataFrame.
        
        # Si el nombre de la columna tiene la etiqueta "fixed" en config["dynamic_limits"], la variable es fija y no debe modificarse
        if config["dynamic_limits"].get(variable_name) == "fixed":  # 
            # Variable fija, los límites son el mismo valor
            X_min.append(value)
            X_max.append(value)
        else:
            # Aplicar los límites dinámicos definidos
            # Obtiene los valores lower (límite inferior) y upper (límite superior) definidos en el diccionario de configuración para la columna actual (variable_name).
            lower, upper = config["dynamic_limits"][variable_name]
            X_min.append(value + lower)
            X_max.append(value + upper)
    
    return np.array(X_min), np.array(X_max)


def generate_individual(random, df, start_index, n_var_independientes, ventana, config):
    """
    Genera un individuo basado en configuraciones personalizadas.
    
    Parameters:
    random (Random): Generador aleatorio.
    df (DataFrame): DataFrame con los datos originales.
    start_index (int): Índice de inicio para seleccionar las filas consecutivas.
    n_var_independientes (int): Número de variables independientes a considerar (columnas).
    ventana (int): Tamaño de la ventana (número de filas a seleccionar).
    config (dict): Configuración personalizada para los límites dinámicos.
    
    Returns:
    individual (array): Array de tamaño `(ventana, n_var_independientes)` con los valores generados.
    """
    individual = []

    # Asegurarse de trabajar solo con las primeras `n_var_independientes` columnas(No debemos incluir la columna Target: En este caso la ultima columna)
    df_independientes = df.iloc[:, :n_var_independientes]  # Selecciona únicamente las primeras n_var_independientes columnas del DataFrame 
    config_2["column_names"] = df_independientes.columns.tolist() # Guarda los nombres de estas columnas en config["column_names"] para usarlos más adelante en la generación de límites dinámicos.

    # Verificar que el índice de inicio + ventana no exceda el número de filas en df_independientes
    if start_index + ventana > len(df_independientes):
        raise IndexError(f"El índice de inicio más la ventana excede el número de filas disponibles en el DataFrame (inicio: {start_index}, ventana: {ventana}).")
    
    # Iterar a través de la ventana
    for i in range(ventana): # Itera sobre las filas consecutivas del DataFrame, comenzando desde start_index
        row = df_independientes.iloc[start_index + i].values
        
        # Generar límites dinámicos para la fila actual
        X_min, X_max = generate_dynamic_limits(row, config = config_2)

        # Generar una fila del individuo respetando los límites
        random_row = [random.uniform(X_min[j], X_max[j]) for j in range(len(row))]
        
        # Añadir la fila generada al individuo
        individual.append(random_row)
    
    return np.array(individual)

# individuo = generate_individual(random, df_prueba, start_index=0, n_var_independientes=14, ventana=window_size, config = config )
# individuo

def fitness_function(candidates, args, modelo_lstm):
    fitness_values = []
    for candidate in candidates:
        # Ajustar la forma del individuo para que sea compatible con la entrada del modelo LSTM
        individuo_reshape = np.array(candidate).reshape(1, 4, 14)
        
        # Realizar la predicción usando el modelo LSTM
        prediccion = modelo_lstm.predict(individuo_reshape)
        
        # Extraer el valor de la predicción (consumo de combustible) y añadirlo a la lista de fitness
        fitness_values.append(prediccion[0][0])  # Asegúrate de que `prediccion` tenga la forma correcta
    return fitness_values

# # Generar la población inicial
# poblacion = generate_population(n_individuos_poblacion=3, random=random, df=datos_genetic, start_index=0, n_var_independientes=14)

# # # Evaluar la función de fitness en la población generada
# fitness_scores = fitness_function(candidates=poblacion, args=None, modelo_lstm=modelo_lstm)
# print(f"Fitness scores: {fitness_scores}")\

# Función de evaluación y algoritmo genético
def run_genetic_algorithm(modelo_lstm, df, n_individuos_poblacion=5, generations=5, start_index=0, window_size=4):
    # Crear el generador de números aleatorios
    prng = Random()
    
    # Número de variables independientes en cada individuo
    n_var_independientes = 14  
    
    # Crear el objeto del algoritmo genético (GA)
    ea = ec.GA(prng)

    # Configurar la función generadora de individuos
    def generator(random, args):
        #return generate_individual(prng, df, start_index, n_var_independientes, window_size)
        #return generate_individual(random, df, start_index=0, n_var_independientes=14, ventana=window_size)
        return generate_individual(random, df, start_index=0, n_var_independientes=14, ventana=window_size, config = config_2)
    
    # Crear un contenedor global para almacenar los fitness de cada generación
    fitness_history = {}

    # Configurar la función de evaluación (fitness)
    def evaluator(candidates, args):
        # Evaluar el fitness de todos los candidatos (individuos)
        fitness_values = fitness_function(candidates, args, modelo_lstm)
        
        # Obtener el número de generación actual
        current_generation = len(fitness_history)
        #print(f"Evaluando Generación {current_generation}...")
        
        # Guardar los fitness de la generación actual
        fitness_history[current_generation] = fitness_values
        
        # Mostrar los fitness de la generación actual
        #print(f"\nFitness de la Generación {current_generation}: {fitness_values}")
        
        return fitness_values

    # Configuración de operadores genéticos
    ea.variator = [
        ec.variators.uniform_crossover,  # Cruce uniforme
        ec.variators.gaussian_mutation   # Mutación gaussiana
    ]

    # Cambiar el método de reemplazo a generational_replacement
    ea.replacer = ec.replacers.generational_replacement
    ea.selector = ec.selectors.tournament_selection  # Selección por torneo

    # Configurar la terminación por número de generaciones
    ea.terminator = ec.terminators.generation_termination

    # Ejecutar la evolución
    final_pop = ea.evolve(
        generator=generator,
        evaluator=evaluator,
        pop_size=n_individuos_poblacion,
        maximize=False, #Problema de Minimizacion 
        num_elites=1,
        max_generations=generations
    )

    # Obtener el mejor individuo
    best_individual = min(final_pop, key=lambda x: x.fitness)
    best_fitness = best_individual.fitness

    print("\nMejor individuo (minimizar consumo de combustible):", best_individual.candidate)
    print("Consumo de combustible predicho (fitness):", best_fitness)

    return best_individual.candidate, best_fitness


# Función para aplicar el algoritmo genético por bloques
def aplicar_algoritmo_genetico_por_bloques(modelo_lstm, resultados, window_size=4, n_individuos_poblacion=5, generations=3):
    # Nombres de columnas originales del DataFrame
    columnas_originales = resultados[0].columns.tolist()  # Aseguramos que incluya todas las columnas de resultados[0]
    
    # Crear un DataFrame vacío para almacenar los resultados
    df_resultados_final = pd.DataFrame(columns=columnas_originales) 
    
    # Iterar sobre los bloques en resultados con una ventana deslizante
    for i in range(len(resultados) - window_size + 1):  # Ajustamos el rango para no exceder el tamaño del DataFrame
        # Ejecutar el algoritmo genético en el bloque actual, pasando el índice de inicio como `start_index`
        best_individual, best_fitness = run_genetic_algorithm(
            modelo_lstm, resultados[i], n_individuos_poblacion=n_individuos_poblacion, generations=generations, start_index=i, window_size=window_size)
        
        # Extraer solo la última fila de best_individual (la última fila de la matriz 4x14)
        ultima_fila = best_individual[-1]  # Esto obtiene la última fila del array
        
        # Crear una nueva fila con los valores de ultima_fila y best_fitness
        nueva_fila = list(ultima_fila)  # Convertimos la última fila en una lista
        nueva_fila.append(best_fitness)  # Agregamos el best_fitness al final
        
        # Convertimos la fila en un DataFrame temporal
        df_temp = pd.DataFrame([nueva_fila], columns=columnas_originales) 
        
        # Anexamos la fila al DataFrame final
        df_resultados_final = pd.concat([df_resultados_final, df_temp], ignore_index=True)

        # Ajustar el índice del DataFrame final para que comience en window_size - 1
        df_resultados_final.index = range(window_size - 1, window_size - 1 + len(df_resultados_final))

    return df_resultados_final


# resultados -> lista de DataFrames que contiene tus bloques de datos
# modelo_lstm -> tu modelo LSTM ya entrenado
df_resultados_recomendacion_vf = aplicar_algoritmo_genetico_por_bloques(modelo_lstm, resultados, window_size=4, n_individuos_poblacion=5, generations=3)

/tmp/ipykernel_7049/2834286069.py:32: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  subset = pd.concat([subset, filas_nan], ignore_index=True)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 167ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step

Mejor individuo (minimizar consumo de combustible): [[ 6.03973719e-01 -3.89703723e-01 -4.24623264e-01  1.92636536e+00
   1.21658886e+00  4.34750267e-01  4.63418121e-01  4.21461594e-01
   2.49181122e+00  2.11949947e+00  2.53508206e-01  2.88046286e-01
  -1.10532031e+00  1.25440002e+02]
 [-1.25196590e+00 -4.14139973e-02 -3.80468462e-01  3.95669018e-01
 

/tmp/ipykernel_7049/2834286069.py:248: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_resultados_final = pd.concat([df_resultados_final, df_temp], ignore_index=True)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step

Mejor individuo (minimizar consumo de combustible): [[-1.27247864e+00  3.66062390e-01  7.90901329e-01  7.59726832e-01
   2.19921673e+00  4.37891114e-01  4.63617400e-01  4.19659086e-01
   2.98362654e+00  2.37844501e+00  6.14766629e-01 -2.65474857e-01
   5.16483602e-01  2.81940002e+02]
 [-1.81155024e+00  8.30928510e-01 -9.63772036e-01  4.84025207e-01
   2.03382323e+00  4.30849829e-01  4.63834266e-01  3.94200972e-01
   2.80546271e+00  1.76833037e+00 -3.38172731e-01 

In [349]:
datos_genetic.head(10)

,Roll,Intake Manifold Pressure,Intake Manifold #2 Pressure,Engine Speed,Actual Gear,x,y,z,efhcargado,efhvacio,distancia_ruta_pendiente_grados,Ambient Air Temperature,Brake Pump Pressure,Fuel Consumption Rate(lag1),Fuel Consumption Rate(target)
0,-0.185613,-0.031118,-0.227759,0.927779,0.254222,0.434750,0.463418,0.421462,2.069348,1.512208,-0.29231,0.125486,-0.305275,125.440002,281.940002
1,-0.638040,-0.020913,-0.098554,0.606561,1.292677,0.437891,0.463617,0.419659,2.069348,1.512208,-0.29231,0.128764,-0.282307,281.940002,0.000000
2,-2.030574,-0.054912,-0.621835,1.192444,1.292677,0.430850,0.463834,0.394201,2.069348,1.512208,-0.29231,0.125486,-0.113067,0.000000,0.000000
3,-0.758491,-0.055875,-0.641055,1.000166,1.811904,0.422928,0.464084,0.342244,2.069348,1.512208,-0.29231,0.124502,-0.291575,0.000000,362.890015
4,0.587039,0.033894,-0.033952,1.163037,1.811904,0.428635,0.464248,0.345397,2.069348,1.512208,-0.29231,0.123301,-0.290366,362.890015,412.940002
5,1.303871,0.264856,2.178688,0.516077,1.811904,0.445958,0.464304,0.383466,2.069348,1.512208,-0.29231,0.119039,-0.281702,412.940002,472.500000
6,1.124663,0.221559,1.792687,1.116287,1.292677,0.452901,0.464494,0.440137,2.069348,1.512208,-0.29231,0.111282,-0.281904,472.500000,203.789993
7,-0.805496,0.052415,0.012885,0.348682,-0.265005,0.453758,0.464678,0.473542,2.069348,1.512208,-0.29231,0.099263,-0.282105,203.789993,377.890015
8,0.219809,-0.029398,-0.308513,1.178871,1.811904,0.451833,0.464922,0.462702,2.069348,1.512208,-0.29231,0.097187,-0.281098,377.890015,26.750000
9,-0.118042,-0.056423,-0.665119,-0.615725,-0.784233,0.454033,0.465043,0.465261,2.069348,1.512208,-0.29231,0.097187,3.657144,26.750000,26.500000


In [367]:
df_resultados_recomendacion_vf

,Roll,Intake Manifold Pressure,Intake Manifold #2 Pressure,Engine Speed,Actual Gear,x,y,z,efhcargado,efhvacio,distancia_ruta_pendiente_grados,Ambient Air Temperature,Brake Pump Pressure,Fuel Consumption Rate(lag1),Fuel Consumption Rate(target)
3,-0.792368,-1.030012,0.264411,0.479118,2.310219,0.422928,0.464084,0.342244,1.139020,2.333369,-0.627610,-0.190359,0.287244,0.000000,36.114605
4,0.668150,0.886908,0.249659,0.355033,0.937731,0.428635,0.464248,0.345397,2.802344,0.613543,0.630144,0.556560,-0.193834,362.890015,152.139999
5,2.097536,-0.654255,2.638298,-0.115539,1.289680,0.445958,0.464304,0.383466,2.655559,1.884766,-1.073774,0.294238,-0.772195,412.940002,247.607025
6,1.011598,-0.019814,0.407671,-0.660826,1.169349,-0.344424,-0.332832,-0.357188,1.656994,1.501196,-1.691276,-0.917337,-0.859385,471.702675,105.760391
7,-1.792921,-0.482466,0.390681,0.087721,0.721228,0.453758,0.464678,0.473542,2.853371,2.307681,-0.880235,0.232572,-1.199082,203.789993,67.213028
8,0.197525,-1.530836,-0.011936,0.553075,2.109015,-0.060257,-0.047168,-0.049387,2.299557,0.831221,-1.728014,-0.444107,-0.880830,377.377925,95.583679
9,-1.029062,-0.562821,-1.409726,-1.197688,-1.254148,0.454033,0.465043,0.465261,2.935770,0.854633,-0.927286,0.340925,3.292228,26.750000,29.859177
10,0.171626,-0.745605,0.165154,-0.020969,-0.034017,0.454054,0.465043,0.465193,2.108107,1.345602,-0.572525,-0.706328,-1.083334,26.500000,29.014561
11,0.064078,0.272883,0.007877,0.609680,-0.428940,0.454225,0.465042,0.466915,1.595114,0.807473,-0.444291,-0.036411,-0.040022,253.389999,81.253075
12,0.439212,-0.041965,0.043344,-1.527209,-1.394601,0.456352,0.465032,0.466376,0.828456,2.220194,-0.465729,-0.850363,0.125508,27.000000,42.197689
